In [1]:
from platform import system
from targets import target_values
from residuals import mdl_resid
from tiktak import filer
 
print('Hi!')
 
import os
os.environ['MKL_CBWR']='AUTO'

from estimates import get_point

Hi!


In [2]:
x_base, tm = get_point(True,read_wisdom=False) # high education

tar = target_values(tm)

x_comp, _ = get_point(False,read_wisdom=False) # low education
print(x_base)
print(x_comp)

{'sigma_psi': 0.3869773077864603, 'sigma_psi_init': 0.9938164169266216, 'pmeet_21': 0.26138291062442615, 'pmeet_30': 0.7186472606306563, 'pmeet_40': 0.6593653665181554, 'preg_21': 0.01355186141662465, 'preg_28': 0.026776949709968387, 'preg_35': 0.037440655536951584, 'u_shift_mar': 0.23416090232520456, 'util_alp': 0.4080762765646524, 'util_kap': 11.265720476636952, 'util_qbar': 30.846312906184785, 'disutil_marry_sm_mal': 16.579301983282175, 'disutil_shotgun': 7.775996065960521, 'abortion_costs': 7.10104593516042, 'p_abortion_access': 0.5697660445900381, 'u_lost_divorce': 13.108545716002517, 'mu_psi_init': -0.5566081252903454, 'high education': True}
{'pmeet_21': 0.728698619306192, 'pmeet_30': 0.23310107046750475, 'pmeet_40': 0.9629955398495929, 'preg_21': 0.03687725755909954, 'preg_28': 0.0743925695472168, 'preg_35': 0.19133138664351215, 'util_qbar': 25.181454069571686, 'disutil_marry_sm_mal': 14.040986098063753, 'disutil_shotgun': 5.8766374053947485, 'abortion_costs': 4.317887395330901

In [3]:
pmeets = ['pmeet_21','pmeet_30','pmeet_40']
ppregs = ['preg_21','preg_28','preg_35']
abortions = ['abortion_costs','p_abortion_access']
prefs = ['disutil_marry_sm_mal','disutil_shotgun','u_lost_divorce','util_qbar']
# form xlist
he = ['high education']
replace_fields = [('trend only',he),
                  ('pmeets',pmeets),
                  ('ppregs',ppregs),
                  ('remar',['disutil_marry_sm_mal']),
                  ('stigma',['disutil_shotgun']),            
                  ('div costs',['u_lost_divorce']),
                  ('qbar',['util_qbar']),
                  ('abortions',abortions),
                  ('all probs',pmeets+ppregs),
                  ('all prefs',prefs),
                  ('prefs and pmeets',prefs+pmeets),
                  ('prefs and ppregs',prefs+ppregs),
                  ('prefs and abortions',prefs+abortions),
                  ('probs and abortions',pmeets+ppregs+abortions),
                  ('trend and all prefs',he+prefs),
                  ('trend and all probs',he+pmeets+ppregs),   
                  ('trend and abortions',he+abortions),
                  ('all but trend',prefs+pmeets+ppregs+abortions)
                 ]

xlist = [x_base]
names = ['college']
for name, rf in replace_fields:
    xnew = x_base.copy()
    for f in rf:
        xnew[f] = x_comp[f]
    xlist.append(xnew)
    names.append(name)
    
names.append('high school')
xlist.append(x_comp)

names = ['educ comparison: '+ n for n in names]
print(names)

['educ comparison: college', 'educ comparison: trend only', 'educ comparison: pmeets', 'educ comparison: ppregs', 'educ comparison: remar', 'educ comparison: stigma', 'educ comparison: div costs', 'educ comparison: qbar', 'educ comparison: abortions', 'educ comparison: all probs', 'educ comparison: all prefs', 'educ comparison: trend and all prefs', 'educ comparison: trend and all probs', 'educ comparison: trend and abortions', 'educ comparison: all but trend', 'educ comparison: high school']


In [12]:
for x, name in zip(xlist,names):        
    print(name)
    #print(x)
    
    out,  mom = mdl_resid(x=x,targets=tar,
                                      return_format=['distance','moments'],
                                      verbose=False,draw=False,cs_moments=False,
                                      save_to ='mdl for {}'.format(name),
                                      moments_save_name = name,
                                      moments_repeat=5)
    
    
    
    
    


educ comparison: college
Divorced in cross-section: 11.1 KF, 4.1 MF, ratio 5.8
educ comparison: trend only
Divorced in cross-section: 7.0 KF, 1.8 MF, ratio 8.4
educ comparison: pmeets
Divorced in cross-section: 12.7 KF, 1.4 MF, ratio 31.5
educ comparison: ppregs
Divorced in cross-section: 8.5 KF, 5.1 MF, ratio 3.1
educ comparison: remar
Divorced in cross-section: 10.2 KF, 4.6 MF, ratio 4.3
educ comparison: stigma
Divorced in cross-section: 11.4 KF, 4.1 MF, ratio 5.9
educ comparison: div costs
Divorced in cross-section: 43.5 KF, 4.7 MF, ratio 18.5
educ comparison: qbar
Divorced in cross-section: 11.4 KF, 4.4 MF, ratio 5.4
educ comparison: abortions
Divorced in cross-section: 11.3 KF, 4.1 MF, ratio 6.0
educ comparison: all probs
Divorced in cross-section: 8.1 KF, 4.9 MF, ratio 8.2
educ comparison: all prefs
Divorced in cross-section: 17.3 KF, 5.4 MF, ratio 5.7
educ comparison: trend and all prefs
Divorced in cross-section: 16.4 KF, 3.1 MF, ratio 9.0
educ comparison: trend and all probs
D

In [19]:
# this converts moments to outputs
names_select = [('Col','educ comparison: college'),
                ('HS','educ comparison: high school'), 
                ('I','educ comparison: trend only'),
                ('M','educ comparison: pmeets'), 
                ('P','educ comparison: ppregs'),
                ('D','educ comparison: div costs'),
                ('R','educ comparison: remar'), 
                ('S','educ comparison: stigma'),
                ('A','educ comparison: abortions'),
                ('Q','educ comparison: qbar')]

tags = ''
div_kfs = 'divorced if kids first'
div_mfs = 'divorced if marriage first'
div_rats = 'divorced ratio KF/MF'
kf_rels = 'relative KF at 35'

for tag, name in names_select:
    mom = filer(name + '.pkl',0,0,repeat=False)
    div_10_kf = mom['divorced by years after marriage if kids first, 5']
    div_10_mf = mom['divorced by years after marriage if marriage first, 5']
    share_kf_35 = mom['k then m in sample at 35']
    div_cs_kf = mom['divorced if k then m and one marriage']
    div_cs_mf = mom['divorced if m then k and one marriage']
    div_kfs += ' & {:02.1f}'.format(100*div_10_kf)
    div_mfs += ' & {:02.1f}'.format(100*div_10_mf)
    div_rats += ' & {:02.1f}'.format(div_10_kf/div_10_mf)
    kf_rels += ' & {:02.1f}'.format(100*share_kf_35)
    tags += (' & ' + tag)
    
tags += r' \\'
div_kfs += r' \\'
div_mfs += r' \\'
div_rats += r' \\'
kf_rels += r' \\'

    
print(tags)
print(div_kfs)
print(div_mfs)
print(div_rats)
print(kf_rels)
    


 & Col & HS & I & M & P & D & R & S & A & Q \\
divorced if kids first & 10.8 & 12.2 & 2.6 & 12.0 & 8.5 & 41.9 & 9.6 & 11.0 & 11.1 & 11.1 \\
divorced if marriage first & 2.1 & 7.6 & 0.9 & 0.4 & 2.9 & 2.5 & 2.5 & 2.1 & 2.1 & 2.3 \\
divorced ratio KF/MF & 5.2 & 1.6 & 3.0 & 28.5 & 3.0 & 16.8 & 3.8 & 5.3 & 5.3 & 4.8 \\
relative KF at 35 & 10.5 & 46.2 & 41.7 & 16.8 & 24.5 & 14.1 & 11.6 & 10.5 & 10.2 & 10.4 \\


In [20]:
# this converts moments to outputs
names_select = [('HS','educ comparison backwards: high school'),
                ('Col','educ comparison backwards: college'), 
                ('I','educ comparison backwards: trend only'),
                ('M','educ comparison backwards: pmeets'), 
                ('P','educ comparison backwards: ppregs'),
                ('D','educ comparison backwards: div costs'),
                ('R','educ comparison backwards: remar'), 
                ('S','educ comparison backwards: stigma'),
                ('A','educ comparison backwards: abortions'),
                ('Q','educ comparison backwards: qbar')]

tags = ''
div_kfs = 'divorced if kids first'
div_mfs = 'divorced if marriage first'
div_rats = 'divorced ratio KF/MF'
kf_rels = 'relative KF at 35'

for tag, name in names_select:
    mom = filer(name + '.pkl',0,0,repeat=False)
    div_10_kf = mom['divorced by years after marriage if kids first, 5']
    div_10_mf = mom['divorced by years after marriage if marriage first, 5']
    share_kf_35 = mom['k then m in sample at 35']
    div_cs_kf = mom['divorced if k then m and one marriage']
    div_cs_mf = mom['divorced if m then k and one marriage']
    div_kfs += ' & {:02.1f}'.format(100*div_10_kf)
    div_mfs += ' & {:02.1f}'.format(100*div_10_mf)
    div_rats += ' & {:02.1f}'.format(div_10_kf/div_10_mf)
    kf_rels += ' & {:02.1f}'.format(100*share_kf_35)
    tags += (' & ' + tag)
    
tags += r' \\'
div_kfs += r' \\'
div_mfs += r' \\'
div_rats += r' \\'
kf_rels += r' \\'

    
print(tags)
print(div_kfs)
print(div_mfs)
print(div_rats)
print(kf_rels)

 & HS & Col & I & M & P & D & R & S & A & Q \\
divorced if kids first & 12.2 & 10.8 & 23.9 & 9.0 & 6.7 & 4.4 & 11.0 & 21.6 & 11.9 & 10.7 \\
divorced if marriage first & 7.6 & 2.1 & 5.5 & 3.6 & 0.8 & 2.0 & 2.3 & 9.6 & 8.0 & 6.7 \\
divorced ratio KF/MF & 1.6 & 5.2 & 4.3 & 2.5 & 8.1 & 2.2 & 4.9 & 2.2 & 1.5 & 1.6 \\
relative KF at 35 & 46.2 & 10.5 & 22.0 & 57.9 & 40.4 & 47.2 & 52.4 & 48.3 & 46.9 & 48.2 \\
